In [297]:
import os
from datetime import datetime
import operator
import re

import numpy as np

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc

plt.rc('text', usetex=True)
rc('font',**{'family':'serif','serif':['Times']})
plt.rcParams['text.latex.preamble'] = [r'\boldmath']

In [263]:
# Searching for the log and error files and storing them into lists
log_folder = "../logs"
folder_files = []

for r, d, f in os.walk(log_folder):
    for file in f:
        if '.txt' in file:
            folder_files.append(os.path.join(r, file))

log_files = []
error_files = []
            
for curr_file in folder_files:
    cf_orig = curr_file
    curr_file = curr_file.split("/")[2].split("_")
    
    cf_type = curr_file[0]
    cf_mode = curr_file[1]
    cf_time = curr_file[2].split(".txt")[0]
    
    cft_obj = datetime.strptime(cf_time, '%Y-%m-%d %H:%M:%S')
    cft_unix = cft_obj.timestamp()
    
    file_info = [cft_unix, cf_mode, cf_orig]
    
    globals()[str(cf_type)+"_files"].append(file_info)

In [585]:
# Looking through the cache log and recognition log files 

def log_parser(cl_name, rl_name, lc_file):    
    # number of lines in file
    with open(cl_name) as f:
        csf_num_lines = sum(1 for _ in f)
    with open(rl_name) as g:
        rsf_num_lines = sum(1 for _ in g)
    with open(lc_file) as g:
        lcf_num_lines = sum(1 for _ in g)
    
    # no of frames dealt with
    no_frames = open(cl_name, 'r').read().count("res sent")
    
    # read file contents of logs
    cs_file = open(cl_name, "r").readlines()
    rs_file = open(rl_name, "r").readlines()
    logcat_file = open(lc_file, "r").readlines()
    
    # storing current frame being considered
    cf_ticker = []
    
    # array for latencies which are in units of ms
    session_latencies = np.zeros([no_frames, 9])
    transfer_latencies = np.zeros([no_frames, 5])
    
    negative_results = []
    
    no_frames_considered = 0
    # looping over cache log
    for cs_line in range(csf_num_lines):
        if not cs_line < 8:
            if len(cf_ticker) == 4:
                cf_ticker = []
                no_frames_considered += 1
            curr_line = cs_file[cs_line]
            if curr_line == "\n":
                pass
            else:
                cl_split = curr_line.split(" ")

                curr_frame = cl_split[1]
                cf_ticker.append(curr_frame)
                
                if len(cf_ticker) == 1:
                    session_latencies[no_frames_considered][0] = curr_frame
                    transfer_latencies[no_frames_considered][0] = curr_frame
                    
                transfer_latencies[no_frames_considered][len(cf_ticker)] = cl_split[-1]
                
                if "res sent, marker#: 0" in curr_line:
                    negative_results.append(curr_frame)
                #print(cl_split)
    
    # storing the index of where the current frame is, in session_latencies
    sl_where = 0
        
    # looping over recognition log
    for rs_line in range(rsf_num_lines):
        if not rs_line < 5:
            curr_line = rs_file[rs_line]
            
            if not curr_line == "\n":
                cl_split = curr_line.split(" ")
            
            if "received, filesize:" in curr_line:
                curr_frame_no = cl_split[1]
                sl_where = np.where(session_latencies[:,0] == float(curr_frame_no))[0][0]
                sift_extract_time = 0
                
            if not curr_frame_no in negative_results:
                if "ExtractSift time total = " in curr_line:
                    indiv_sift_time = float(re.search("ExtractSift time total =(.*)ms", curr_line).group(1))
                    sift_extract_time += indiv_sift_time

                if "sift res" in curr_line:
                    sift_time = float(cl_split[-1]) * 1000
                    session_latencies[sl_where][3] = sift_time

                    #print(sift_extract_time, sift_time)

                if "pca encoding time" in curr_line:
                    pca_time = float(cl_split[-1]) * 1000
                    session_latencies[sl_where][4] = pca_time

                if "fisher encoding time" in curr_line:
                    fsh_time = float(cl_split[-1]) * 1000
                    session_latencies[sl_where][5] = fsh_time

                if "MatchSiftData time" in curr_line:
                    msd_time = float(cl_split[-2]) 
                    session_latencies[sl_where][6] = msd_time

                if "FindHomography time" in curr_line:
                    fhy_time = float(cl_split[-2])
                    session_latencies[sl_where][7] = fhy_time

    # Client times
    pp_begin_uxtime = 0
    frame_sent_ux_time = 0 
    curr_frame_no = 0
    curr_frame_loc = 0
    
    # offset of phone in seconds
    android_offset = 3.330
    
    for lg_line in range(lcf_num_lines):
        curr_line = logcat_file[lg_line]
        cl_split = curr_line.split(" ")
        
        if "get gray scaled frame data at" in curr_line:
            pp_begin_uxtime = float(cl_split[-1])
        
        if "sent with size" in curr_line:
            # find frame number
            frame_no = float(re.search("frame(.*)sent with size", curr_line).group(1))
            
            if not frame_no in negative_results:
                # check for existence of frame number in server results
                sl_where = np.where(session_latencies[:,0] == frame_no)[0]

                if not sl_where.size == 0:
                    curr_frame_no = frame_no
                    curr_frame_loc = sl_where[0]

                    frame_sent_ux_time = float(cl_split[-1]) 

                    pp_time = frame_sent_ux_time - pp_begin_uxtime
                    session_latencies[curr_frame_loc][1] = pp_time

                    server_rec = transfer_latencies[curr_frame_loc][1]
                    data_transfer = abs(server_rec - (frame_sent_ux_time/1000) - android_offset)
                    #print((frame_sent_ux_time/1000),android_offset, server_rec)
                    session_latencies[curr_frame_loc][2] = data_transfer

        if "res received at" in curr_line:
            client_rec = (float(cl_split[-1]) / 1000)

        if "OMX-VDEC-1080P: Video slvp perflock acquired" in curr_line:
            curr_date = cl_split[0]
            curr_year = "19"

            curr_time = cl_split[1]
            dt_string = curr_year + "-" + curr_date + " " + curr_time
            curr_dt = datetime.strptime(dt_string, '%y-%m-%d %H:%M:%S.%f')
            cdt_unix = curr_dt.timestamp()

            client_pp = cdt_unix - client_rec
            session_latencies[curr_frame_loc][8] = client_pp
                 
    # Delete non-positive results rows
    for nr_i in range(len(negative_results)):
        curr_nr = negative_results[nr_i]
        cnr_loc = np.where(session_latencies[:,0] == float(curr_nr))[0][0]
        session_latencies = np.delete(session_latencies, cnr_loc, 0)
                
    return session_latencies
        

In [586]:
# Parsing log files 

server_mode = {
    '1': "recognition server",
    '0': "cache server"
}

# Sort by UNIX timestamp in ascending order
log_files = sorted(log_files, key=operator.itemgetter(0))

# for cf_i in range(len(log_files)):
#     if cf_i % 2 == 0:
#         # only consider pairs of files together

recog_server = log_files[-2]
cache_server = log_files[-1]

print("Current two log files are:")
print(recog_server, cache_server)

rs_file = recog_server[2]
cs_file = cache_server[2]

# perform two checks to make sure correct files are being used
rs_mode = recog_server[1]
cs_mode = cache_server[1]

if not (rs_mode == "1") and not (cs_mode == "0"):
    print("An error occurred in servers start up, remove either following files to correct:")
    print(rs_file)
    print(cs_file)
    pass

rs_time = recog_server[0]
cs_time = cache_server[0]

if not (cs_time - rs_time) < 10:
    print("Servers were started too late apart - start them within 10 s of each other")
    pass

# Collecting logcat file from Android Studio output
logcat_file = "/home/jacky/Desktop/mobile_ar_system/ar_apps/cloudridar_movieapp_client/logcat.txt"

# Send file names to parser
parsed_results = log_parser(cs_file, rs_file, logcat_file)

tasks = {
    1 : "Client pre-processing: ",
    2 : "Data transfer: ",
    3 : "SIFT feature extraction: ",
    4 : "PCA dimension reduction: ",
    5 : "FV encoding with GMM: ",
    6 : "LSH NN searching: ",
    7 : "Template matching: ",
    8 : "Client post-processing: "
}

print("\n")

overall_latency_med = 0
overall_latency_std = 0
for i in range(len(parsed_results[:,0])):
    curr_index = i + 1 
    if curr_index <= len(tasks):
        med_val = np.median(parsed_results[:,curr_index])
        std_val = np.std(parsed_results[:,curr_index])

        overall_latency_med += med_val
        overall_latency_std += std_val

        print(tasks[curr_index] + str(med_val) + " +- " + str(std_val))

print("Overall latency: " + str(overall_latency_med) + " +- " + str(std_val))

Current two log files are:
[1570093332.0, '1', '../logs/log_1_2019-10-03 12:02:12.txt'] [1570093335.0, '0', '../logs/log_0_2019-10-03 12:02:15.txt']


Client pre-processing: 12.5 +- 4.199999999999999
Data transfer: 6.65553505897522 +- 0.039528036751684835
SIFT feature extraction: 3.53193283081055 +- 1.3141686848760223
PCA dimension reduction: 3.59857082366943 +- 0.9434871648887053
FV encoding with GMM: 3.40092182159424 +- 0.8647845739730776
LSH NN searching: 1.275 +- 0.16930445948054648
Template matching: 4.665 +- 0.6511405378257447
Client post-processing: 1.0830000638961792 +- 0.01390103476069095
Overall latency: 36.709960598945614 +- 0.01390103476069095
